# 최대 유량 (Maximum Flow)

`-` 간선에 용량이 존재하는 방향 그래프에서 소스에서 싱크까지 보낼 수 있는 최대 유량을 계산하자

## 도시 왕복하기 1

- 문제 출처: [백준 17412번](https://www.acmicpc.net/problem/17412)

`-` 네트워크 플로우를 사용하는 문제이다

`-` 생소한 내용이라 관련 자료를 많이 찾아봤다

`-` 참고: https://en.wikipedia.org/wiki/Flow_network

`-` 그래프 내에 증가 경로가 없을 때까지 유량을 흘려보내서 최대 유량을 계산할 수 있다

`-` 소스에서 싱크까지 흘려보낼 수 있는 최대 유량을 $f$라 하고 노드의 개수를 $V$, 간선의 개수를 $E$라고 하자

`-` 매 증가 경로마다 적어도 $1$의 유량을 흘리므로 증가 경로를 DFS로 찾는다면 시간 복잡도는 $O((V + E)f)$가 된다

`-` 그런데 보통 $V\le E$이므로 단순하게 $O(Ef)$라고 표현한다

`-` 이때 증가 경로를 BFS로 찾는 방식을 Edmonds–Karp 알고리즘이라고 한다

`-` Edmonds–Karp 알고리즘의 시간 복잡도는 $\min\left(O(Ef), O\left(VE^2\right)\right)$이다

`-` 참고: https://en.wikipedia.org/wiki/Edmonds-Karp_algorithm

`-` 찾아본 자료를 보면 보통 코드가 길던데 개념적으로 접근하면 그리 복잡하진 않다

`-` 증가 경로를 찾기 위해 BFS를 사용하는데 BFS는 기본 그래프 알고리즘이다

`-` 증가 경로가 존재하는지 판단하는 건 소스에서 출발해 싱크까지 조건하에 BFS로 도달 가능한지 따지는 것이다

`-` 여기서 조건이란 간선의 유량이 간선의 용량을 초과하지 않는 것이다

`-` 증가 경로를 찾았으면 해당 경로를 따라 유량을 흘려보낸다

`-` 증가 경로 추적을 위해 역추적 배열을 만들고 BFS 안에서 갱신해야 된다

`-` 증가 경로 중 일부에 실제로는 존재하지 않는 역방향 간선이 있을 수 있다

`-` 역방향 간선은 흘러온 유량을 다시 되돌려 보낸다고 생각하면 된다

`-` 어떤 간선 $u\to v$를 통해 유량을 보냈다면 반대로 $v \to u$에는 그만큼의 유량을 차감해줘야 한다 (유량의 대칭성)

`-` 아무튼 증가 경로가 존재하지 않을 때까지 반복하면 최대 유량을 계산할 수 있다

`-` 이제 본 문제를 풀어보자

`-` $N$개의 도시가 $P$개의 단방향 길로 연결되어 있으니 방향 그래프로 구현해주면 된다

`-` $1$번에서 $2$번으로 가는 서로 다른 경로를 최대한 많이 찾아야 하는데 한 경로에 포함된 길이 다른 경로에 포함되면 안된다

`-` 이는 간선의 용량이 $1$인 것과 마찬가지다

`-` 따라서 모든 간선의 용량을 $1$로 설정한 후 소스를 $1$번 도시, 싱크를 $2$번 도시로 취급해 Edmonds–Karp 알고리즘을 사용하여 최대 유량을 계산하면 된다

`-` 최대 유량을 찾는 과정만 제대로 알고 있다면 구현은 비교적 간단하다

`-` 간선의 용량이 $1$이라 증가 경로에 있는 모든 간선들의 잔여 용량의 최솟값인 $\operatorname{bottleneck}$은 항상 $1$이지만 연습 차원에서 일반화된 코드로 작성했다

In [8]:
from collections import defaultdict, deque


def edmonds_karp(graph, source, sink, flow, capacity):
    max_flow = 0
    exists_augmenting_path = True
    while exists_augmenting_path:
        prevs = bfs(graph, source, sink, flow, capacity)
        exists_augmenting_path = prevs[sink] != NONE
        if not exists_augmenting_path:
            break
        augmenting_path = track_augmenting_path(prevs, sink)
        bottleneck = compute_bottleneck(augmenting_path, flow, capacity)
        update_flow(augmenting_path, flow, bottleneck)
        max_flow += bottleneck
    return max_flow


def bfs(graph, source, sink, flow, capacity):
    queue = deque([source])
    visited = {source: True}
    prevs = [NONE] * (N + 1)
    while queue:
        u = queue.popleft()
        for v in graph[u]:
            residual_capacity = capacity[u][v] - flow[u][v]
            if v in visited or residual_capacity <= 0:
                continue
            prevs[v] = u
            if v == sink:
                return prevs
            visited[v] = True
            queue.append(v)
    return prevs


def track_augmenting_path(prevs, sink):
    augmenting_path = []
    node = sink
    while node != NONE:
        augmenting_path.append(node)
        node = prevs[node]
    augmenting_path.reverse()
    return augmenting_path


def compute_bottleneck(augmenting_path, flow, capacity):
    bottleneck = INF
    for i in range(len(augmenting_path) - 1):
        u, v = augmenting_path[i], augmenting_path[i + 1] 
        bottleneck = min(capacity[u][v] - flow[u][v], bottleneck)
    return bottleneck


def update_flow(augmenting_path, flow, bottleneck):
    for i in range(len(augmenting_path) - 1):
        u, v = augmenting_path[i], augmenting_path[i + 1] 
        flow[u][v] += bottleneck
        flow[v][u] -= bottleneck


def solution():
    global N, NONE, INF
    N, P = map(int, input().split())
    INF = float("inf")
    NONE = -1
    source = 1
    sink = 2
    flow = [[0] * (N + 1) for _ in range(N + 1)]
    capacity = [[1] * (N + 1) for _ in range(N + 1)]
    graph = defaultdict(list)
    for _ in range(P):
        start, end = map(int, input().split())
        graph[start].append(end)  # 정방향
        graph[end].append(start)  # 역방향
        capacity[end][start] = 0  # 역방향 간선의 용량은 0
    max_flow = edmonds_karp(graph, source, sink, flow, capacity)
    print(max_flow)


solution()

# input
# 5 5
# 1 3
# 3 2
# 1 5
# 5 4
# 4 2

 5 5
 1 3
 3 2
 1 5
 5 4
 4 2


2
